<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 7:** Retrieval-Augmented Generation with Vector Stores</font>

<br>

In the previous notebook, we learned about embedding models and exercised some of their capabilities. We discussed their intended use cases of longer-form document comparison and found ways to use it as a backbone for more custom semantic comparisons. This notebook will progress these ideas toward the retrieval model's intended use case and explore how to build chatbot systems that rely on *vector stores* to automatically save and retrieve information.

<br>

### **Learning Objectives:**

- Understand how semantic-similarity-backed systems can facilitate easy-to-use retrieval formulations.

- Learn how to incorporate retrieval modules into your chat model systems for a retrieval-augmented generation (RAG) pipeline, which can be applied to tasks like document retrieval and conversation memory buffers.

<br>

### **Questions To Think About:**

- This notebook does not attempt to incorporate hierarchical reasoning or non-naive RAG (such as planning agents). Consider what modifications would be necessary to make these components work in an LCEL chain.

- Consider when it would be best to move your vector store solution into a scalable service and when a GPU will become necessary for optimization.

<br>

### **Environment Setup:**

In [1]:
# %%capture
## ^^ Comment out if you want to see the pip install process

## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

----

<br>

## Part 1: Summary of RAG Workflows

This notebook will explore several paradigms and derive reference code to help you approach some of the most common retrieval-augmented workflows. Specifically, the following sections will be covered (with the differences highlighted):

<br>

> ***Vector Store Workflow for Conversational Exchanges:***
- Generate semantic embedding for each new conversation.
- Add the message body to a vector store for retrieval.
- Query the vector store for relevant messages to fill in the LLM context.

<br>

> ***Modified Workflow for an Arbitrary Document:***
- **Divide the document into chunks and process them into useful messages.**
- Generate semantic embedding for each **new document chunk**.
- Add the **chunk bodies** to a vector store for retrieval.
- Query the vector store for relevant **chunks** to fill in the LLM context.
    - ***Optional:* Modify/synthesize results for better LLM results.**

<br>

> **Extended Workflow for a Directory of Arbitrary Documents:**
- Divide **each document** into chunks and process them into useful messages.
- Generate semantic embedding for each new document chunk.
- Add the chunk bodies to **a scalable vector database for fast retrieval**.
    - ***Optional*: Exploit hierarchical or metadata structures for larger systems.**
- Query the **vector database** for relevant chunks to fill in the LLM context.
    - *Optional:* Modify/synthesize results for better LLM results.

<br>

Some of the most important terminology surrounding RAG is covered in detail on the [**LlamaIndex Concepts page**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html), which itself is a great starting point for progressing towards the LlamaIndex loading and retrieving strategy. We highly recommend using it as a reference as you continue with this notebook and advise you to try out LlamaIndex after the course to consider the pros and cons firsthand!


<!-- > <img src="https://drive.google.com/uc?export=view&id=1cFbKbVvLLnFPs3yWCKIuzXkhBWh6nLQY" width=1200px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/data_connection_langchain.jpeg" width=1200px/>
>
> From [**Retrieval | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/)

----

<br>

## **Part 2:** RAG for Conversation History

In our previous explorations, we delved into the capabilities of document embedding models and used them to embed, store, and compare semantic vector representations of text. Though we could motivate how to efficiently extend this into vector store land manually, the true beauty of working with a standard API is its strong incorporation with other frameworks that can already do the heavy lifting for us!

<br>

### **Step 1**: Getting A Conversation

Consider a conversation crafted using Llama-13B between a chat agent and a blue bear named Beras. This dialogue, dense with details and potential diversions, provides a rich dataset for our study:


In [3]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

Using the manual embedding strategy from the previous notebook is still very viable, but we can also rest easy and let a **vector store** do all that work for us!

<br>

### **Step 2:** Constructing Our Vector Store Retriever

To streamline similarity queries on our conversation, we can employ a vector store to help keep track of passages for us! **Vector Stores**, or vector storage systems, abstract away most of the low-level details of the embedding/comparison strategies and provide a simple interface to load and compare vectors.


<!-- > <img src="https://drive.google.com/uc?export=view&id=1ZjwYbSZzsXK6ZP8O1-cY3BeRffV4oqzb" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/vector_stores.jpeg" width=1200px/>
>
> From [**Vector Stores | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/)

<br>

In addition to simplifying the process from an API perspective, vector stores also implement connectors, integrations, and optimizations under the hood. In our case, we will start with the [**FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss), which integrates a LangChain-compatable Embedding model with the [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss) library to make the process fast and scalable on our local machine!

**Specifically:**

1. We can feed our conversation into [**a FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss) via the `from_texts` constructor. This will take our conversational data and the embedding model to create a searchable index over our discussion.
2. This vector store can then be "interpreted" as a retriever, supporting the LangChain runnable API and returning documents retrieved via an input query.

The following shows how you can construct a FAISS vector store and reinterpret it as a retriever using the LangChain `vectorstore` API:

In [4]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 220 ms, sys: 414 ms, total: 634 ms
Wall time: 8.72 s


The retriever can now be used like any other LangChain runnable to query the vector store for some relevant documents:

In [5]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        id='b3378a38-b6ad-4cbe-bdc8-deae87eadccd',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='f102d070-bb0e-4993-89be-ac31c4cdf064',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        id='859adb9d-17a7-4aef-b3a6-f3fda3051c75',
        metadata={},
        page_content='[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for 
you![Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.'
    ),
    Document(
        id='94a16deb-6604-47c7-9cb4-533b40296150',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    )
]

In [6]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        id='682fcf43-6a27-48f5-91ae-1931d0c1f272',
        metadata={},
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        id='94a16deb-6604-47c7-9cb4-533b40296150',
        metadata={},
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        id='b3378a38-b6ad-4cbe-bdc8-deae87eadccd',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='f102d070-bb0e-4993-89be-ac31c4cdf064',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

As we can see, our retriever found a handful of semantically relevant documents from our query. You may notice that not all of the documents are useful or clear on their own. For example, a retrieval of *"Beras"* for *"your name"* may be problematic for the chatbot if provided out of context. Anticipating the potential problems and creating synergies between your LLM components can increase the likelihood of good RAG behavior, so keep an eye out for such pitfalls and opportunities.

<br>

### **Step 3:** Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [7]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [8]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    # | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

It seems that Beras lives in the Arctic. The cooler climate there is quite different from the warm climate found in
the Rocky Mountains, which Beras mentioned was of interest.

Take a second to try out some more invocations and see how the new setup performs. Regardless of your model choice, the following questions should serve as interesting starting points.

In [9]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. You can enjoy 
their majestic beauty from here. I hope that helps! Let me know if you'd like to learn more about them.

In [10]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

Hello there! The Rocky Mountains are a stunning range of mountains that stretch across North America. To answer 
your question about their location, they run from northern British Columbia in Canada all the way down to New 
Mexico in the United States. As for their proximity to California, they don't directly border the state. The 
closest parts of the Rocky Mountains to California would be in Nevada or Idaho, which are still quite a distance 
away. I hope that helps! Let's continue exploring more about the Rocky Mountains if you have any other questions.

In [11]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

While I can't provide specifics on Beras's location, I can tell you that the Rocky Mountains span from western 
Canada all the way down to the southwestern United States. Without knowing Beras's exact location in the Arctic, I 
can't determine the precise distance. However, given that Beras lives in the Arctic, they would be quite far from 
the Rocky Mountains.

<br>

You might notice some decent performance with this always-on retrieval node in the loop since the actual context being fed into the LLM remains relatively small. It's important to experiment with factors like embedding sizes, context limits, and model options to see what kinds of behavior you can expect and which efforts are worth taking to improve performance.

<br>

### **Step 4:** Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

# instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

While I'm sure the Rocky Mountains would provide a magnificent backdrop for enjoying some ice cream, the context 
only mentions the scenic beauty and significance of the Rockies, not any food options. However, we can certainly 
agree that ice cream is a delicious treat, and perhaps imagining enjoying it amidst the breathtaking views of the 
Rockies would make it even more enjoyable!

While I can't guess your favorite food with absolute certainty, based on our conversation, I can tell that you seem
to have a liking for ice cream! I'm sure enjoying ice cream amidst the magnificent Rocky Mountains would be a 
wonderful experience. Is ice cream one of your favorite foods?

Well, that's quite a delightful surprise! Honey is such a versatile and beneficial food, with a rich, sweet taste 
that's hard to resist. In our conversation, you had mentioned enjoying ice cream, and I had associated that with 
your favorite food. But it turns out, your favorite is the all-natural, golden sweetness of honey! I'm glad we got 
that cleared up, and now I know a bit more about your preferences. Thank you for sharing that with me!

Absolutely, I do! After our little chat, I found out that your favorite food is indeed honey. I apologize for the 
misunderstanding earlier, but I'm glad we got that cleared up. Honey really is a wonderful choice with its unique 
taste and natural benefits. Thanks again for sharing that with me!

Unlike the more automatic full-text or rule-based approaches to injecting context into the LLM, this approach ensures some amount of consolidation which can keep the context length from getting out of hand. It's still not a full-proof strategy on its own, but it's a stark improvement for unstructured conversations (and doesn't even require a strong instruction-tuned model to perform slot-filling).

----

<br>

## **Part 3 [Exercise]:** RAG For Document Chunk Retrieval

Given our prior exploration of document loading, the idea that data chunks can be embedded and searched through probably isn't surprising. With that said, it is definitely worth going over since applying RAG with documents is a double-edged sword; it may **seem** to work well out of the box but requires some extra care when optimizing it for truly reliable performance. It also provides an excellent opportunity to review some fundamental LCEL skills, so let's see what we can do!

<br>

### **Exercise:**

In the previous example, you may recall that we pulled in some relatively small papers with the help of [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv) using the following syntax:

```python
from langchain.document_loaders import ArxivLoader

docs = [
    ArxivLoader(query="2205.00445").load(),  ## MRKL
    ArxivLoader(query="2210.03629").load(),  ## ReAct
]
```

Given all that you've learned so far, choose a selection of papers that you would like to use and develop a chatbot that can talk about them!

<br>

Though this is a pretty big task, a walkthrough of ***most*** of the process will be provided below. By the end of the walkthrough, many of the necessary puzzle pieces will be provided, and your real task will be to integrate them together for the final `retrieval_chain`. When you're done, get ready to re-integrate the chain (or a flavor of your choice) in the last notebook as part of the evaluation exercise!


<br>

### **Task 1**: Loading And Chunking Your Documents

The following code block gives you some default papers to load in for your RAG chain. Feel free to select more papers as desired, but note that longer documents will take longer to process. A few simplifying assumptions and additional processing steps are included to help you improve your naive RAG performance:

- Documents are cut off prior to the "References" section if one exists. This will keep our system from considering the citations and appendix sections, which tend to be long and distracting.

- A chunk that lists the available documents is inserted to provide a high-level view of all available documents in a single chunk. If your pipeline does not provide metadata on each retrieval, this is a useful component and can even be listed among a list of higher-priority pieces if appropriate.

- Additionally, the metadata entries are also inserted to provide general information. Ideally, there would also be some synthetic chunks that merge the metadata into interesting cross-document chunks.

**NOTE:** ***For the sake of the assessment, please include at least one paper that is less than one month old!***


In [13]:
import json
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [
    ArxivLoader(query="2508.02836").load(),  ## Agentic Privacy-Preserving Machine Learning (July 30, 2025)
    ArxivLoader(query="2508.02994").load(),  ## When AIs Judge AIs: The Rise of Agent-as-a-Judge Evaluation for LLMs (Aug 2025)
    ArxivLoader(query="2508.02979").load(),  ## Unified Tool Integration for LLMs: A Protocol-Agnostic Approach (Aug 2025)
    ArxivLoader(query="2507.09089").load(),  ## Measuring the Impact of Early-2025 AI on Experienced Open-Source Developer Productivity (July 2025)
    ArxivLoader(query="2507.02554").load(),  ## AI Research Agents for Machine Learning: Search, Exploration, and Generalization (July 3, 2025)
    
    # ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    # ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    # ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    # ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    # ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    # ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ## Some longer papers
    # ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    # ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    # ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
    ## TODO: Feel free to add more
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Agentic Privacy-Preserving Machine Learning
 - When AIs Judge AIs: The Rise of Agent-as-a-Judge Evaluation for LLMs
 - Unified Tool Integration for LLMs: A Protocol-Agnostic Approach to Function Calling
 - Measuring the Impact of Early-2025 AI on Experienced Open-Source Developer Productivity
 - AI Research Agents for Machine Learning: Search, Exploration, and Generalization in MLE-bench 

Document 0
 - # Chunks: 31
 - Metadata: 


{
    'Published': '2025-07-30',
    'Title': 'Agentic Privacy-Preserving Machine Learning',
    'Authors': 'Mengyu Zhang, Zhuotao Liu, Jingwen Huang, Xuanqi Liu',
    'Summary': 'Privacy-preserving machine learning (PPML) is critical to ensure data privacy\nin AI. Over the past
few years, the community has proposed a wide range of\nprovably secure PPML schemes that rely on various 
cryptography primitives.\nHowever, when it comes to large language models (LLMs) with billions of\nparameters, the 
efficiency of PPML is everything but acceptable. For instance,\nthe state-of-the-art solution for confidential LLM 
inference represents at\nleast 10,000-fold slower performance compared to plaintext inference. The\nperformance gap
is even larger when the context length increases. In this\nposition paper, we propose a novel framework named 
Agentic-PPML to make PPML in\nLLMs practical. Our key insight is to employ a general-purpose LLM for 
intent\nunderstanding and delegate cryptographically secure inference to specialized\nmodels trained on vertical 
domains. By modularly separating language intent\nparsing - which typically involves little or no sensitive 
information - from\nprivacy-critical computation, Agentic-PPML completely eliminates the need for\nthe LLMs to 
process the encrypted prompts, enabling practical deployment of\nprivacy-preserving LLM-centric services.'
}


Document 1
 - # Chunks: 78
 - Metadata: 


...

{
    'Published': '2025-08-05',
    'Title': 'Unified Tool Integration for LLMs: A Protocol-Agnostic Approach to Function Calling',
    'Authors': 'Peng Ding, Rick Stevens',
    'Summary': 'The proliferation of tool-augmented Large Language Models (LLMs) has created\na fragmented 
ecosystem where developers must navigate multiple protocols,\nmanual schema definitions, and complex execution 
workflows. We address this\nchallenge by proposing a unified approach to tool integration that abstracts\nprotocol 
differences while optimizing execution performance. Our solution\ndemonstrates how protocol-agnostic design 
principles can significantly reduce\ndevelopment overhead through automated schema generation, dual-mode 
concurrent\nexecution, and seamless multi-source tool management. Experimental results show\n60-80% code reduction 
across integration scenarios, performance improvements up\nto 3.1x through optimized concurrency, and full 
compatibility with existing\nfunction calling standards. This work contributes both theoretical insights\ninto tool
integration architecture and practical solutions for real-world LLM\napplication development.'
}


Document 3
 - # Chunks: 46
 - Metadata: 


{
    'Published': '2025-07-25',
    'Title': 'Measuring the Impact of Early-2025 AI on Experienced Open-Source Developer Productivity',
    'Authors': 'Joel Becker, Nate Rush, Elizabeth Barnes, David Rein',
    'Summary': 'Despite widespread adoption, the impact of AI tools on software development\nin the wild remains 
understudied. We conduct a randomized controlled trial\n(RCT) to understand how AI tools at the February-June 2025 
frontier affect the\nproductivity of experienced open-source developers. 16 developers with moderate\nAI experience
complete 246 tasks in mature projects on which they have an\naverage of 5 years of prior experience. Each task is 
randomly assigned to allow\nor disallow usage of early 2025 AI tools. When AI tools are allowed, 
developers\nprimarily use Cursor Pro, a popular code editor, and Claude 3.5/3.7 Sonnet.\nBefore starting tasks, 
developers forecast that allowing AI will reduce\ncompletion time by 24%. After completing the study, developers 
estimate that\nallowing AI reduced completion time by 20%. Surprisingly, we find that allowing\nAI actually 
increases completion time by 19%--AI tooling slowed developers\ndown. This slowdown also contradicts predictions 
from experts in economics (39%\nshorter) and ML (38% shorter). To understand this result, we collect and\nevaluate 
evidence for 20 properties of our setting that a priori could\ncontribute to the observed slowdown effect--for 
example, the size and quality\nstandards of projects, or prior developer experience with AI tooling. Although\nthe 
influence of experimental artifacts cannot be entirely ruled out, the\nrobustness of the slowdown effect across our
analyses suggests it is unlikely\nto primarily be a function of our experimental design.'
}


Document 4
 - # Chunks: 51
 - Metadata: 


{
    'Published': '2025-07-03',
    'Title': 'AI Research Agents for Machine Learning: Search, Exploration, and Generalization in MLE-bench',
    'Authors': 'Edan Toledo, Karen Hambardzumyan, Martin Josifoski, Rishi Hazra, Nicolas Baldwin, Alexis 
Audran-Reiss, Michael Kuchnik, Despoina Magka, Minqi Jiang, Alisia Maria Lupidi, Andrei Lupu, Roberta Raileanu, 
Kelvin Niu, Tatiana Shavrina, Jean-Christophe Gagnon-Audet, Michael Shvartsman, Shagun Sodhani, Alexander H. 
Miller, Abhishek Charnalia, Derek Dunfield, Carole-Jean Wu, Pontus Stenetorp, Nicola Cancedda, Jakob Nicolaus 
Foerster, Yoram Bachrach',
    'Summary': "AI research agents are demonstrating great potential to accelerate scientific\nprogress by 
automating the design, implementation, and training of machine\nlearning models. We focus on methods for improving 
agents' performance on\nMLE-bench, a challenging benchmark where agents compete in Kaggle competitions\nto solve 
real-world machine learning problems. We formalize AI research agents\nas search policies that navigate a space of 
candidate solutions, iteratively\nmodifying them using operators. By designing and systematically 
varying\ndifferent operator sets and search policies (Greedy, MCTS, Evolutionary), we\nshow that their interplay is
critical for achieving high performance. Our best\npairing of search strategy and operator set achieves a 
state-of-the-art result\non MLE-bench lite, increasing the success rate of achieving a Kaggle medal from\n39.6% to 
47.7%. Our investigation underscores the importance of jointly\nconsidering the search strategy, operator design, 
and evaluation methodology in\nadvancing automated machine learning."
}

<br>

### **Task 2**: Construct Your Document Vector Stores

Now that we have all of the components, we can go ahead and create indices surrounding them:

In [14]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 598 ms, sys: 17.9 ms, total: 616 ms
Wall time: 20 s


<br>

From there, we can combine our indices into a single one using the following utility:

In [15]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 253 chunks


<br>

### **Task 3: [Exercise]** Implement Your RAG Chain

Finally, all the puzzle pieces are in place to implement the RAG pipeline! As a review, we now have:

- A way to construct a from-scratch vector store for conversational memory (and a way to initialize an empty one with `default_FAISS()`)

- A vector store pre-loaded with useful document information from our `ArxivLoader` utility (stored in `docstore`).

With the help of a couple more utilities, you're finally ready to integrate your chain! A few additional convenience utilities are provided (`doc2str` and the now-common `RPrint`) but are optional to use. Additionally, some starter prompts and structures are also defined.

> **Given all of this:** Please implement the `retrieval_chain`.

In [16]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

################################################################################################
## BEGIN TODO: Implement the retrieval chain to make your system work!

# Long context reorder utility
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

retrieval_chain = (
    {'input': (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({
        'history': itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str
    })
    | RunnableAssign({
        'context': itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str  
    })
)

## END TODO
################################################################################################

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about RAG!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Tell me about RAG!\n\n From this, we have retrieved the following potentially-useful info:  
Conversation History Retrieval:\n\n\n Document Retrieval:\n[Quote from AI Research Agents for Machine Learning: 
Search, Exploration, and Generalization in MLE-bench] ., 2025), a benchmark\\nwhere AI agents compete in Kaggle 
competitions\\n1o1-preview is deprecated.\\n1\\n(1)\\u00a0\\nSelect 
Node(s)\\n(3)\\nEvaluation\\nSearch\\nCode\\n1\\n2\\n3\\n4\\n5\\n6\\n1\\n2\\n3\\n4\\n5\\n6\\n7\\n1\\n2\\n3\\n4\\n5\
\n6\\nOperators\\n...\\nDraft\\nDebug\\nImprove\\nScore\\nInvalid\\nLow\\nHigh\\n\\u00a0\\nState\\n(2)\\u00a0\\nSel
ect & Apply\\nOperator\\nRepeat\\n1\\n2\\n3\\n4\\n5\\n6\\n7\\nFigure 2 Overview of AIRA. Given a problem 
specification, AIRA maintains a search graph whose nodes are (partial)\\nsolutions. At each iteration, the agent 
(1) selects nodes via a selection policy, (2) picks an operator via an operator\\npolicy and applies this operator 
to the node, and (3) scores the resulting solution via a fitness function. Here, a greedy\\nnode-selection strategy
applies the improve operator to the highest-scoring node.\\nto solve real-world machine learning (ML) problems. 
Notably, the state-of-the-art approach AIDE (Jiang et al\n[Quote from AI Research Agents for Machine Learning: 
Search, Exploration, and Generalization in MLE-bench] . Notably, the state-of-the-art approach AIDE (Jiang et 
al.,\\n2025) does not fully disentangle these performance factors, providing limited insight into which 
components\\nprimarily drive the agent\\u2019s performance and where improvements are needed.\\nWe start by 
formalizing the design of AI research agents as a search algorithm composed of two components: The\\nfirst one is 
the search policy which is used to navigate the space of candidate solutions, and the second is the\\noperators 
which iteratively modifies existing solutions to generate new candidate solutions. In this framework\\n(Fig. 2), 
AIDE is represented as a greedy search algorithm that, at each step, applies one of its code operators\\n(i.e., 
Draft, Debug, Improve) to the current best solution. This allows us to disentangle the effect of the\\nsearch from 
that of the operators\n[Quote from AI Research Agents for Machine Learning: Search, Exploration, and Generalization
in MLE-bench] .\\nThis combination of \\u03c0sel, \\u03c0op, and OAIDE defines the baseline agent we denote 
AIDEgreedy (also summarized\\nin Appendix B). When the agent uses an alternative search policy while keeping OAIDE 
unchanged, we write\\nAIDEsearch_policy to identify change. For example, AIDEmcts uses the AIDE operator set and 
MCTS as the\\nsearch policy.\\n3\\nExperiment Design\\n3.1\\nAIRA-dojo\\nAn agent\\u2019s environment greatly 
influences its performance. To systematically study the space of agentic policies\\n(see Fig. 7), we introduce the 
AI Research Agent (AIRA) dojo\\u2014a scalable and customizable framework for AI\\nresearch agents. AIRA-dojo 
provides the environment in which agents operate, along with abstractions for\\noperators and policies as described
in Section 2, and tasks that define evaluation criteria for agent performance\n[Quote from AI Research Agents for 
Machine Learning: Search, Exploration, and Generalization in MLE-bench] . Superimage, a container image, provides a
base set of tools required for common ML tasks.\\nThis image includes pre-configured CUDA support, key deep 
learning frameworks such as PyTorch and\\nTensorFlow, and essential data science libraries. Each coding session 
starts from the original Superimage\\nstate and can diverge based on the agent\\u2019s actions while not affecting 
the state of the other agents.\\nTogether, these design choices create a stable and robust testbed that eliminates 
confounders at both the\\nsyst

RAG seems to refer to a concept from the document "AI Research Agents for Machine Learning: Search, Exploration, and Generalization in MLE-bench" from the year 2025. However, the document does not provide a clear definition or explanation of RAG on its own. I had to make some assumptions based on the context provided.

From what I can gather, RAG might be an acronym for "Reinforcement Agent in Graph," which is a hypothetical concept introduced in the document. This concept is used to describe a type of AI research agent that operates within a search graph, where nodes represent (partial) solutions to a given problem.

In this context, RAG maintains a search graph whose nodes are candidate solutions, and at each iteration, the agent selects nodes via a selection policy, picks an operator via an operator policy, and scores the resulting solution via a fitness function.

The document mentions RAG as a part of the AIRA (AI Research Agent) framework, which is designed to solve real-world ma

### **Task 4:** Interact With Your Gradio Chatbot

In [ ]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

/usr/local/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.41.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4160273c296fb96210.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: tell me a cool fact\n\n From this, we have retrieved the following potentially-useful info:  
Conversation History Retrieval:\n[Quote from Document] Agent previously responded with RAG seems to refer to a 
concept from the document "AI Research Agents for Machine Learning: Search, Exploration, and Generalization in 
MLE-bench" from the year 2025. However, the document does not provide a clear definition or explanation of RAG on 
its own. I had to make some assumptions based on the context provided.\n\nFrom what I can gather, RAG might be an 
acronym for "Reinforcement Agent in Graph," which is a hypothetical concept introduced in the document. This 
concept is used to describe a type of AI research agent that operates within a search graph, where nodes represent 
(partial) solutions to a given problem.\n\nIn this context, RAG maintains a search graph whose nodes are candidate 
solutions, and at each iteration, the agent selects nodes via a selection policy, picks an operator via an operator
policy, and scores the resulting solution via a fitness function.\n\nThe document mentions RAG as a part of the 
AIRA (AI Research Agent) framework, which is designed to solve real-world machine learning problems by searching 
and iteratively modifying existing solutions to generate new candidate solutions.\n\nSources:\n\n* AI Research 
Agents for Machine Learning: Search, Exploration, and Generalization in MLE-bench. (2025). Retrieved from [Quote 
from AI Research Agents for Machine Learning: Search, Exploration, and Generalization in MLE-bench]\n[Quote from 
Document] User previously responded with Tell me about RAG!\n\n\n Document Retrieval:\n[Quote from When AIs Judge 
AIs: The Rise of Agent-as-a-Judge Evaluation for LLMs] .\\n\\u2022 Integrating Tool Use in Judges: Agent 
judges\\nthat can use tools (like search engines, code\\ninterpreters, calculators) will become more\\nimportant as
tasks get complex or require live\\ndata verification. We might see hybrid evalua-\\ntor agents: for example, an 
agent that, when\\nevaluating a factual answer, automatically\\nconsults a knowledge base or the web to 
fact-\\ncheck statements. This would dramatically\\nimprove evaluations of factual correctness and\\nreduce 
hallucination going unnoticed. Tool-\\nusing judges could also simulate user inter-\\naction \\u2013 e.g\n[Quote 
from Measuring the Impact of Early-2025 AI on Experienced Open-Source Developer Productivity] . Appendix G provides
more detail about our\\nrecruitment and incentivization process.\\nThe repositories themselves are large and 
mature. On average, they have 23,000 stars, 1,100,000\\nlines of code, 4,900 forks, 20,000 commits, and 710 
committers, and they broadly have very high\\nquality bars for code contributions. For example, one set of 
repository contribution guidelines con-\\ncludes: \\u201cPhew. While the above may be a lot to remember [..] the 
motivation for enforcing process\\nis to ensure that all code contributions meet a certain quality 
threshold.\\u201d Section G.7 details further\\nstatistics about individual developers and 
repositories.\\n4\\nFigure 3: Real issues completed during the study from the stdlibjs and mito 
repositories\\n2.2\\nExperimental Design\\nEach developer provides a list of real issues in their repository to 
work on as part of this study.\\nIssues are typically bug reports, feature requests, or work items used to 
coordinate development\n[Quote from AI Research Agents for Machine Learning: Search, Exploration, and 
Generalization in MLE-bench] . Superimage, a container image, provides a base set of tools required for common ML 
tasks.\\nThis image includes pre-configured CUDA support, key deep learning frameworks such as PyTorch 
and\\nTensorFlow, and essential data science libraries. Each coding session 

<br>

----

<br>

## **Part 4:** Saving Your Index For Evaluation

After you've implemented your RAG chain, please save your accumulated vector store as shown [in the official documentation](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading). You'll have a chance to use it again for your final assessment!

In [18]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


If everything was properly saved, the following line can be invoked to pull the index from the compressed `tgz` file (assuming the pip requirements are installed). After you have confirmed that the cell can pull in your index, download `docstore_index.tgz` for use in the last notebook!

In [19]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
. Results shown are for agents using R1. Bars depict the absolute\nperformance gap between three configurations: (i) using the validation metric for both intermediate (search) and final\n(submission) node selection; (ii) using the validation metric only for search and the test metric for selection; and (iii)\nusing the test metric for both search and selection. b) Bridging the validation\u2013test gap. Medal rate achieved by two\nfinal node selection strategies as a function of k: (i) randomly sampling k nodes and reporting the highest test score\namong them; (ii) selecting the top k nodes by validation score and reporting the highest test score among those. As k\nincreases, the validation-based strategy closes the gap to the upper-bound performance given by the best test score\nover the entire search graph.\npattern. We observe that the rankings between agents change over time. For example, at the 3-hour mark,\nthe pe

-----

<br>

## **Part 5:** Wrap-Up

Congratulations! Assuming your RAG chain is all good, you're now ready to move on to the **RAG Evaluation [Assessment]** section!

### <font color="#76b900">**Great Job!**</font>

### **Next Steps:**
1. **[Optional]** Revisit the **"Questions To Think About" Section** at the top of the notebook and think about some possible answers.

---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>